# Webscraping

In [52]:
# Import necessary libraries
import requests # make HTTP requests to fetch web pages content
from bs4 import BeautifulSoup # parse HTML and XML docs for easier data extraction
import csv # write scraped data into CSV file
import time # introduce delays between requests to avoid server overload
import re #regular expression
import random # vary time delays to simulate human-like behaviour
import psycopg2 #py package to interact with PostgreSQL
from psycopg2 import sql
import logging
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('jumia_scraping.log'),
        logging.StreamHandler()  # Also show logs in console
    ]
)

In [53]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('jumia_scraping.log'),
        logging.StreamHandler()  # Also show logs in console
    ]
)

**Webscraping Page: Jumia**

_Products to scrape:_  
1. Tvs 
    * Smart
    * Digital
2. Cookers

In [54]:
# request url
url1 = 'https://www.jumia.co.ke/televisions/#catalog-listing' # TVs url
url2 = 'https://www.jumia.co.ke/home-cooking-appliances-cookers/#catalog-listing' # Cookers url

# User-Agent headers for automating requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


Check url status

In [55]:
# Sending GET requests to each URL and check response
def check_response(urls, headers):
    # Output the status code of each response
    for url in urls:
        try:
            response =requests.get(url, headers=headers)
            print(f"url: {response.url} - status: { response.status_code}")

            if response.status_code != 200:
                print(f"Failed to retrieve page. Status code: {response.status_code}")
                return 1  # Return 1 if the request fails
        except requests.exceptions.RequestException as e:
            # Handle GET request exceptions (timeout, connection/http errors etc.)
            print(f"An error occured: {e}")
            return 1  # Return 1 if an error occurs during the request
        
    return 0  # Return 0 if no error

# List of urls
urls = [url1, url2]
responses = check_response(urls, headers=headers) # Check all url GET requests responses

# Check result
if responses == 0:
    print("All pages retrieved successfully.")
else:
    print("Some pages failed to load.")


url: https://www.jumia.co.ke/televisions/#catalog-listing - status: 200
url: https://www.jumia.co.ke/home-cooking-appliances-cookers/#catalog-listing - status: 200
All pages retrieved successfully.


Considering that the webpages are paginated, there is need to navigatethrough, and retrieve data from each page.
The last page in the page numbers is identified, then all pages are iterated.

The pagination URL parameters...

In [56]:
# Define function to get the last page number
def get_last_page_number(urls, headers):
    """
    get the last page number in a catalog listing for pagination iteration

    args:
        urls: webpage URL or URLs
        headers: User-Agent headers
    """
    # Parse webpage content with BeautifulSoup
    #for url in urls:
    response =requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find link for last page using its aria-label attribute
    last_page_link = soup.find('a', attrs={'aria-label': 'Last Page'})

    if last_page_link and 'href' in last_page_link.attrs:
        last_page_url =last_page_link['href']
        try:
            page_number = last_page_url.split('?page=')[1].split('#')[0]
            return int(page_number)
        except Exception as e:
            print(f"Error extracting last page number: {e}")
            return 1  # Default to 1 if error occurs


In [57]:
def scrape_product_details(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all product articles
        product_articles = soup.find_all('article', {'class': 'prd _fb col c-prd'})
        
        logging.info(f"Found {len(product_articles)} products on {url}")
        
        products = []
        for article in product_articles:
            try:
                # Find the core link which contains most product data
                core_link = article.find('a', {'class': 'core'})
                if not core_link:
                    continue

                # Extract data from data-ga4 attributes which contain the most reliable info
                item_id = core_link.get('data-ga4-item_id', "N/A")
                name = core_link.get('data-ga4-item_name', "N/A")
                brand = core_link.get('data-ga4-item_brand', "N/A")
                price = core_link.get('data-ga4-price', "N/A")
                discount = core_link.get('data-ga4-discount', "N/A")
                category = core_link.get('data-ga4-item_category', "N/A")
                subcategory = core_link.get('data-ga4-item_category2', "N/A")
                
                # Get product URL
                product_url = core_link.get('href')
                if product_url:
                    product_url = f"https://www.jumia.co.ke{product_url}"
                else:
                    product_url = "N/A"

                # Extract displayed price and old price
                price_elem = article.find('div', {'class': 'prc'})
                displayed_price = price_elem.text.strip() if price_elem else "N/A"
                
                old_price_elem = article.find('div', {'class': 'old'})
                old_price = old_price_elem.text.strip() if old_price_elem else "N/A"

                # Extract rating information
                stars_elem = article.find('div', {'class': 'stars _s'})
                reviews_count = "0"
                rating = "N/A"
                
                if stars_elem:
                    rating_text = stars_elem.text.strip()
                    if 'out of 5' in rating_text:
                        rating = rating_text.split('out of 5')[0].strip()
                    
                    # Find review count
                    rev_elem = article.find('div', {'class': 'rev'})
                    if rev_elem:
                        reviews_text = rev_elem.text
                        reviews_count = reviews_text.strip('()') if '(' in reviews_text else "0"

                # Check if it's from official store
                is_official = bool(article.find('div', {'class': 'bdg _mall _xs'}))

                # Check if express shipping is available
                has_express = bool(article.find('svg', {'class': 'ic xprss'}))

                if item_id != "N/A":  # Only add products with valid IDs
                    products.append({
                        "product_name": name,
                        "price": displayed_price,
                        "price_value": price,  # Numerical price value
                        "old_price": old_price,
                        "discount_percentage": discount,
                        "rating": rating,
                        "reviews_count": reviews_count,
                        "item_id": item_id,
                        "item_brand": brand,
                        "product_url": product_url,
                        "category": category,
                        "subcategory": subcategory,
                        "is_official_store": is_official,
                        "has_express_shipping": has_express
                    })
                    
            except Exception as e:
                logging.warning(f"Error extracting product details: {str(e)}")
                continue
                
        logging.info(f"Successfully extracted {len(products)} valid products")
        return products
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed for {url}: {str(e)}")
        return []
    except Exception as e:
        logging.error(f"Unexpected error scraping {url}: {str(e)}")
        return []

In [58]:
# Function to get the last page number from the website using the 'Last Page' link
def get_last_page_number(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the 'Last Page' link using its aria-label attribute
    last_page_link = soup.find('a', attrs={'aria-label': 'Last Page'})
    
    if last_page_link and 'href' in last_page_link.attrs:
        last_page_url = last_page_link['href']
        try:
            page_number = last_page_url.split('?page=')[1].split('#')[0]
            return int(page_number)
        except Exception as e:
            print(f"Error extracting last page number: {e}")
            return 1  # Default to 1 if error occurs
    else:
        print("Last page link not found.")
        return 1  # Default to 1 page if no last page is found


In [59]:
def save_to_postgresql(products, db_params, table_name):
    try:
        # Connect to PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        logging.info(f"Attempting to insert {len(products)} products into {table_name}")
        
        # Create temporary table matching exact schema
        temp_table = f"temp_{table_name}_{int(time.time())}"
        cursor.execute(f"""
            CREATE TEMP TABLE {temp_table} (
                product_name text,
                price text,
                old_price text,
                discount text,
                rating text,
                item_id text,
                item_brand text,
                product_url text
            )
        """)
        
        # Prepare data for insertion
        insert_data = []
        for product in products:
            # Format data to match schema exactly
            row = (
                product["product_name"],
                product["price"],
                product["old_price"],
                product["discount_percentage"],
                f"{product['rating']} ({product['reviews_count']} reviews)",
                product["item_id"],
                product["item_brand"],
                product["product_url"]
            )
            insert_data.append(row)
        
        # Batch insert into temp table
        cursor.executemany(f"""
            INSERT INTO {temp_table} 
            (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, insert_data)
        
        # Insert only new records from temp table to target table
        insert_query = f"""
            INSERT INTO {table_name}
            SELECT t.*
            FROM {temp_table} t
            LEFT JOIN {table_name} m ON m.item_id = t.item_id
            WHERE m.item_id IS NULL
        """
        cursor.execute(insert_query)
        
        # Get counts for logging
        cursor.execute(f"SELECT COUNT(*) FROM {temp_table}")
        temp_count = cursor.fetchone()[0]
        
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        final_count = cursor.fetchone()[0]
        
        # Get count of newly inserted records
        cursor.execute(f"""
            SELECT COUNT(*) FROM {table_name} m
            WHERE EXISTS (
                SELECT 1 FROM {temp_table} t
                WHERE t.item_id = m.item_id
            )
        """)
        new_records = cursor.fetchone()[0]
        
        conn.commit()
        logging.info(f"Processed {temp_count} products for {table_name}")
        logging.info(f"Inserted {new_records} new records into {table_name}")
        logging.info(f"Table {table_name} now has {final_count} total records")
        
    except Exception as e:
        logging.error(f"Database error while processing {table_name}: {str(e)}")
        if conn:
            conn.rollback()
        raise
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [60]:
def scrape_and_save(url, headers, db_params, table_name):
    print(f"\nStarting scrape for {table_name}")
    
    try:
        # Get last page number
        last_page = get_last_page_number(url, headers)
        print(f"Found {last_page} pages to scrape")
        
        # Initialize products list
        all_products = []
        
        # Scrape each page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num}/{last_page}")
            
            # Scrape products from current page
            page_products = scrape_product_details(page_url, headers)
            print(f"Found {len(page_products)} products on page {page_num}")
            all_products.extend(page_products)
            
            # Random delay
            time.sleep(random.uniform(1, 3))
        
        print(f"Total products scraped: {len(all_products)}")
        
        # Verify first product data before saving
        if all_products:
            print("\nSample of first product data:")
            print(json.dumps(all_products[0], indent=2))
        
        # Save products to database
        if all_products:
            print(f"\nAttempting to save {len(all_products)} products to {table_name}")
            save_to_postgresql(all_products, db_params, table_name)
        else:
            print("No products found to save!")
            
    except Exception as e:
        print(f"Error in scrape_and_save: {str(e)}")
        raise

def save_to_postgresql(products, db_params, table_name):
    conn = None
    cursor = None
    try:
        # Connect to PostgreSQL database
        print("Connecting to database...")
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        print(f"Connected successfully. Inserting {len(products)} products...")
        
        # Simple insert query
        insert_query = f"""
            INSERT INTO {table_name} 
            (product_name, price, old_price, discount, rating, item_id, item_brand, product_url)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # Insert each product and commit after each successful insert
        for i, product in enumerate(products, 1):
            try:
                row = (
                    product["product_name"],
                    product["price"],
                    product["old_price"],
                    product["discount_percentage"],
                    f"{product['rating']} ({product['reviews_count']} reviews)",
                    product["item_id"],
                    product["item_brand"],
                    product["product_url"]
                )
                
                cursor.execute(insert_query, row)
                conn.commit()  # Commit after each insert
                
                if i % 100 == 0:
                    print(f"Inserted {i} products...")
                
            except Exception as e:
                print(f"Error inserting product {i}: {str(e)}")
                print(f"Problem product data: {json.dumps(product, indent=2)}")
                continue
        
        # Verify final count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        final_count = cursor.fetchone()[0]
        print(f"\nFinal count in {table_name}: {final_count}")
        
        # Show sample of inserted data
        print("\nVerifying inserted data...")
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 1")
        sample = cursor.fetchone()
        if sample:
            print("Sample record:", sample)
        else:
            print("No records found in database!")
        
    except Exception as e:
        print(f"Database error: {str(e)}")
        if conn:
            conn.rollback()
        raise
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("Database connection closed.")

# Main execution
if __name__ == "__main__":
    try:
        # Your headers and db_params here
        
        # Test database connection first
        print("Testing database connection...")
        conn = psycopg2.connect(**db_params)
        conn.close()
        print("Database connection successful!")
        
        # URLs
        url1 = "https://www.jumia.co.ke/televisions/"
        url2 = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"
        
        # Scrape and save data for both URLs
        scrape_and_save(url1, headers, db_params, "jumia_televisions")
        scrape_and_save(url2, headers, db_params, "jumia_cookers")
        
    except Exception as e:
        print(f"Main execution error: {str(e)}")

Testing database connection...
Database connection successful!

Starting scrape for jumia_televisions
Found 50 pages to scrape
Scraping page 1/50


2025-01-10 20:25:48,041 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=1#catalog-listing
2025-01-10 20:25:48,047 - INFO - Successfully extracted 40 valid products


Found 40 products on page 1
Scraping page 2/50


2025-01-10 20:25:50,412 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=2#catalog-listing
2025-01-10 20:25:50,422 - INFO - Successfully extracted 40 valid products


Found 40 products on page 2
Scraping page 3/50


...

Found 40 products on page 48
Scraping page 49/50


2025-01-10 20:28:58,076 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=49#catalog-listing
2025-01-10 20:28:58,088 - INFO - Successfully extracted 40 valid products


Found 40 products on page 49
Scraping page 50/50


2025-01-10 20:29:00,279 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=50#catalog-listing
2025-01-10 20:29:00,284 - INFO - Successfully extracted 40 valid products


Found 40 products on page 50
Total products scraped: 1960

Sample of first product data:
{
  "product_name": "HTC4388FS - 43\" Smart  Android Frameless TV - Black (1YR WRTY)",
  "price": "KSh 19,459",
  "price_value": "145.87",
  "old_price": "KSh 28,599",
  "discount_percentage": "68.52",
  "rating": "3.9",
  "reviews_count": "3.9 out of 5(2436",
  "item_id": "VI505EA0KTU37NAFAMZ",
  "item_brand": "Vitron",
  "product_url": "https://www.jumia.co.ke/vitron-htc4388fs-43-smart-android-frameless-tv-black-1yr-wrty-93348943.html",
  "category": "Electronics",
  "subcategory": "Television & Video",
  "is_official_store": true,
  "has_express_shipping": true
}

Attempting to save 1960 products to jumia_televisions
Connecting to database...
Connected successfully. Inserting 1960 products...
Error inserting product 1: duplicate key value violates unique constraint "jumia_televisions_item_id_key"
DETAIL:  Key (item_id)=(VI505EA0KTU37NAFAMZ) already exists.

Problem product data: {
  "product_nam

In [61]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import time
import random
import logging
import json  # Add this import

def scrape_and_save(url, headers, db_params, table_name):
    print(f"\nStarting scrape for {table_name}")
    
    try:
        # Get last page number
        last_page = get_last_page_number(url, headers)
        print(f"Found {last_page} pages to scrape")
        
        # Initialize products list
        all_products = []
        
        # Scrape each page
        for page_num in range(1, last_page + 1):
            page_url = f"{url}?page={page_num}#catalog-listing"
            print(f"Scraping page {page_num}/{last_page}")
            
            # Scrape products from current page
            page_products = scrape_product_details(page_url, headers)
            print(f"Found {len(page_products)} products on page {page_num}")
            all_products.extend(page_products)
            
            # Random delay
            time.sleep(random.uniform(1, 3))
        
        print(f"Total products scraped: {len(all_products)}")
        
        # Save products to database
        if all_products:
            print(f"\nFirst product data (sample):")
            first_product = all_products[0]
            for key, value in first_product.items():
                print(f"{key}: {value}")
                
            print(f"\nAttempting to save {len(all_products)} products to {table_name}")
            save_to_postgresql(all_products, db_params, table_name)
        else:
            print("No products found to save!")
            
    except Exception as e:
        print(f"Error in scrape_and_save: {str(e)}")
        raise


In [62]:
def save_to_postgresql(products, db_params, table_name):
    conn = None
    cursor = None
    try:
        # Connect to PostgreSQL database
        print("Connecting to database...")
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        
        # Create table if it doesn't exist
        create_table_query = f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                id SERIAL PRIMARY KEY,
                product_name TEXT,
                price TEXT,
                old_price TEXT,
                discount TEXT,
                rating TEXT,
                reviews_count TEXT,
                category TEXT,
                subcategory TEXT,
                is_official_store BOOLEAN,
                has_express_shipping BOOLEAN,
                item_id TEXT UNIQUE,
                item_brand TEXT,
                product_url TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table {table_name} created or verified")
        
        print(f"Connected successfully. Inserting {len(products)} products...")
        
        # Simple insert query
        insert_query = f"""
            INSERT INTO {table_name} 
            (product_name, price, old_price, discount, rating, reviews_count, 
             category, subcategory, is_official_store, has_express_shipping,
             item_id, item_brand, product_url)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (item_id) DO NOTHING
        """
        
        # Insert each product and commit after each successful insert
        for i, product in enumerate(products, 1):
            try:
                row = (
                    product["product_name"],
                    product["price"],
                    product["old_price"],
                    product["discount_percentage"],
                    product["rating"],
                    product["reviews_count"],
                    product.get("category", "N/A"),  # New fields with default values
                    product.get("subcategory", "N/A"),
                    product.get("is_official_store", False),
                    product.get("has_express_shipping", False),
                    product["item_id"],
                    product["item_brand"],
                    product["product_url"]
                )
                
                cursor.execute(insert_query, row)
                conn.commit()  # Commit after each insert
                
                if i % 100 == 0:
                    print(f"Inserted {i} products...")
                
            except Exception as e:
                print(f"Error inserting product {i}: {str(e)}")
                print(f"Problem product data: {json.dumps(product, indent=2)}")
                continue
        
        # Verify final count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        final_count = cursor.fetchone()[0]
        print(f"\nFinal count in {table_name}: {final_count}")
        
        # Show sample of inserted data
        print("\nVerifying inserted data...")
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 1")
        sample = cursor.fetchone()
        if sample:
            print("Sample record:", sample)
        else:
            print("No records found in database!")
        
    except Exception as e:
        print(f"Database error: {str(e)}")
        if conn:
            conn.rollback()
        raise
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
        print("Database connection closed.")


In [63]:
# Main execution
if __name__ == "__main__":
    try:
        # Test database connection first
        print("Testing database connection...")
        conn = psycopg2.connect(**db_params)
        conn.close()
        print("Database connection successful!")
        
        # URLs
        url1 = "https://www.jumia.co.ke/televisions/"
        url2 = "https://www.jumia.co.ke/home-cooking-appliances-cookers/"
        
        # Scrape and save data for both URLs
        scrape_and_save(url1, headers, db_params, "jumia_televisions")
        scrape_and_save(url2, headers, db_params, "jumia_cookers")
        
    except Exception as e:
        print(f"Main execution error: {str(e)}")

Testing database connection...
Database connection successful!

Starting scrape for jumia_televisions
Found 50 pages to scrape
Scraping page 1/50


2025-01-10 20:29:06,165 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=1#catalog-listing
2025-01-10 20:29:06,175 - INFO - Successfully extracted 40 valid products


Found 40 products on page 1
Scraping page 2/50


2025-01-10 20:29:08,190 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=2#catalog-listing
2025-01-10 20:29:08,198 - INFO - Successfully extracted 40 valid products


Found 40 products on page 2
Scraping page 3/50


...

Found 40 products on page 48
Scraping page 49/50


2025-01-10 20:32:04,135 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=49#catalog-listing
2025-01-10 20:32:04,155 - INFO - Successfully extracted 40 valid products


Found 40 products on page 49
Scraping page 50/50


2025-01-10 20:32:06,240 - INFO - Found 40 products on https://www.jumia.co.ke/televisions/?page=50#catalog-listing
2025-01-10 20:32:06,256 - INFO - Successfully extracted 40 valid products


Found 40 products on page 50
Total products scraped: 2000

First product data (sample):
product_name: HTC4388FS - 43" Smart  Android Frameless TV - Black (1YR WRTY)
price: KSh 19,459
price_value: 145.87
old_price: KSh 28,599
discount_percentage: 68.52
rating: 3.9
reviews_count: 3.9 out of 5(2436
item_id: VI505EA0KTU37NAFAMZ
item_brand: Vitron
product_url: https://www.jumia.co.ke/vitron-htc4388fs-43-smart-android-frameless-tv-black-1yr-wrty-93348943.html
category: Electronics
subcategory: Television & Video
is_official_store: True
has_express_shipping: True

Attempting to save 2000 products to jumia_televisions
Connecting to database...
Table jumia_televisions created or verified
Connected successfully. Inserting 2000 products...
Error inserting product 1: column "reviews_count" of relation "jumia_televisions" does not exist
LINE 3: ...product_name, price, old_price, discount, rating, reviews_co...
                                                             ^

Problem product data: {
 